# Author: Muhammed Sinan

In [ ]:
# Load Libraries

In [ ]:
import tensorflow as tf
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow.keras.backend as K

In [ ]:
#Resizing images is optional, CNNs are ok with large images
SIZE_X = 512 #Resize images (height  = X, width = Y)
SIZE_Y = 512

In [ ]:
#Capture training image info as a list
train_images = []

In [ ]:
for directory_path in glob.glob("/home/sinan/Master_thesis/python_project/new_data/images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)

In [ ]:
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

In [ ]:
train_images = train_images/255.0

In [ ]:
#Capture mask/label info as a list
train_masks = [] 

In [ ]:
for directory_path in glob.glob("/home/sinan/Master_thesis/python_project/new_data/masks/"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X))
        train_masks.append(mask)

In [ ]:
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
train_masks = train_masks/255.0

In [ ]:
#Use customary x_train and y_train variables
X = train_images.astype(np.float)
Y = train_masks.astype(np.float)
#Y = np.expand_dims(Y, axis=3) #May not be necessary.. leftover from previous code 

In [ ]:
#splitting into train and test
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=10)

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((SIZE_X , SIZE_Y, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="relu")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
#Matrics
@tf.autograph.experimental.do_not_convert
def Precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision_keras = true_positives / (predicted_positives + K.epsilon())
    return precision_keras

@tf.autograph.experimental.do_not_convert
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall_keras = true_positives / (possible_positives + K.epsilon())
    return recall_keras

@tf.autograph.experimental.do_not_convert
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
#loss
@tf.autograph.experimental.do_not_convert
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator / denominator

@tf.autograph.experimental.do_not_convert
def f1_score_loss(y_true, y_pred):
    f1_loss= K.binary_crossentropy(y_true, y_pred)+(1 - f1_score(y_true, y_pred))
    return f1_loss

In [ ]:
from tensorflow.keras.optimizers import Adam

adam= tf.keras.optimizers.Adam(
    learning_rate=1e-3,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False)

In [ ]:

import segmentation_models as sm

dice_loss = sm.losses.DiceLoss()
# focal_loss = sm.losses.BinaryFocalLoss()
# total_loss = dice_loss + (1 * focal_loss)
Jaccards_loss = sm.losses.JaccardLoss()

In [ ]:
model = UNet()
#model.load_weights("/home/sinan/Master_thesis/python_project/saved_model/saved_model")
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
#model.summary()

In [ ]:
# model = UNet()
# model.compile(optimizer="adam",
#     loss=f1_score_loss, 
#     metrics=["accuracy", tf.keras.metrics.MeanIoU(num_classes=2)])

In [ ]:
#callbacks = [
        #tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'),
        #tf.keras.callbacks.TensorBoard(log_dir='logs')]
history=model.fit(x_train, 
          y_train,
          batch_size=8, 
          epochs=20,
          verbose=1,
          #callbacks=callbacks,       
          validation_data=(x_val, y_val))

In [ ]:
#accuracy = model.evaluate(x_val, y_val)
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.savefig('/home/sinan/Master_thesis/python_project/graphs/C4_Binary_cross_entropy_with_aug_MIoU_Aug_9.png',dpi=1200)

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['accuracy']
val_loss = history.history['val_accuracy']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training accuracy')
plt.plot(epochs, val_loss, 'r', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
#plt.savefig('/home/sinan/Master_thesis/python_project/graphs/C4_Binary_cross_entropy_accuracy_with_aug_MIoU_Aug_9.png',dpi=1200)


In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['mean_io_u']
val_loss = history.history['val_mean_io_u']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training MIoU')
plt.plot(epochs, val_loss, 'r', label='Validation MIoU')
plt.title('Training and validation MIoU')
plt.xlabel('Epochs')
plt.ylabel('MIoU')
plt.legend()
#plt.savefig('/home/sinan/Master_thesis/python_project/graphs/C4_Binary_cross_entropy_MIoU_with_aug_MIoU_Aug_9.png',dpi=1200)


In [ ]:
test_img = cv2.imread('/home/sinan/Master_thesis/python_project/sinan/train/train1.jpeg', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (SIZE_Y, SIZE_X))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
plt.imshow(test_img, cmap='gray')
test_img = np.expand_dims(test_img, axis=0)

prediction = model.predict(test_img)

#plt.savefig('/home/sinan/Master_thesis/python_project/prediction_image/train_15.png',dpi=1200)

In [ ]:
mask=(512,512)

In [ ]:
#View and Save segmented image
prediction_image = prediction.reshape(mask)
plt.imshow(prediction_image,cmap='gray')
#plt.imsave('images/test_images/segmented.jpg', prediction_image, cmap='gray')
#plt.savefig('/home/sinan/Master_thesis/python_project/prediction_image/predicted_15.png',dpi=1200)


In [ ]:
#model.save_weights("/home/sinan/Master_thesis/python_project/saved_model/saved_model")

In [ ]:
#model.save('/home/sinan/Master_thesis/python_project/models/C4_Binary_cross_entropy_with_MIoU_aug_Aug_9.h5')
#model.save("saved_model/")

In [ ]:
# from tensorflow import keras
# model = keras.models.load_model('/home/sinan/Master_thesis/python_project/models/C4_Binary_cross_entropy_with_aug_Aug_9.hdf5', compile=False)

In [ ]:
# #View and Save segmented image
# prediction_image = prediction.reshape(shape_mask)
# plt.imshow(prediction_image, cmap='gray')